# Retention Optimization with Dynamic Pricing

## Overview
Optimal deductible (retention) selection across insurance market cycles.  We sweep retention levels under soft, normal, and hard markets, compare ergodic vs. ensemble growth perspectives, and simulate multi-year market-cycle transitions.

- **Prerequisites**: [optimization/01_optimization_overview](01_optimization_overview.ipynb)
- **Estimated runtime**: 2-3 minutes
- **Audience**: [Practitioner]

In [ ]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/optimization'

    os.chdir(NOTEBOOK_DIR)
    if NOTEBOOK_DIR not in sys.path:
        sys.path.append(NOTEBOOK_DIR)

    !pip install git+https://github.com/AlexFiliakov/Ergodic-Insurance-Limits.git -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore")

from ergodic_insurance import ManufacturerConfig, InsuranceProgram, EnhancedInsuranceLayer
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.loss_distributions import ManufacturingLossGenerator

plt.style.use("seaborn-v0_8-darkgrid")

# Reproducibility
SEED = 42
np.random.seed(SEED)

## 1. Manufacturing Company Setup

Configure the manufacturer and loss generator used throughout the notebook.

In [ ]:
config = ManufacturerConfig(
    initial_assets=10_000_000,
    asset_turnover_ratio=1.2,
    base_operating_margin=0.10,
    tax_rate=0.25,
    retention_ratio=0.7,
)
manufacturer = WidgetManufacturer(config)
expected_revenue = float(manufacturer.calculate_revenue())

loss_generator = ManufacturingLossGenerator(
    attritional_params={"base_frequency": 4.0, "severity_mean": 30_000, "severity_cv": 0.6},
    large_params={"base_frequency": 0.4, "severity_mean": 400_000, "severity_cv": 0.8},
    catastrophic_params={"base_frequency": 0.02, "severity_xm": 3_000_000, "severity_alpha": 2.0},
    seed=SEED,
)

# Estimate expected annual loss
annual_losses = [
    loss_generator.generate_losses(1.0, expected_revenue)[1]["total_amount"]
    for _ in range(1_000)
]
EAL = float(np.mean(annual_losses))
print(f"Expected revenue  : ${expected_revenue:>12,.0f}")
print(f"Expected ann. loss: ${EAL:>12,.0f}")
print(f"Loss ratio        : {EAL / expected_revenue:>12.2%}")

## 2. Retention Sweep Across Market Cycles

Sweep the deductible (retention) from $25k to $500k under three market conditions.
For each retention level we run a short Monte Carlo to estimate ROE and ruin probability.

In [ ]:
def create_layers(deductible, premium_mult=1.0):
    """Two-layer tower above the given deductible."""
    return InsuranceProgram([
        EnhancedInsuranceLayer(
            attachment_point=int(deductible),
            limit=2_000_000,
            base_premium_rate=0.015 * premium_mult,
        ),
        EnhancedInsuranceLayer(
            attachment_point=int(deductible) + 2_000_000,
            limit=5_000_000,
            base_premium_rate=0.005 * premium_mult,
        ),
    ])

MARKET_MULTS = {"Soft": 0.80, "Normal": 1.00, "Hard": 1.40}
RETENTIONS = np.linspace(25_000, 500_000, 12).astype(int)
N_SIMS, N_YEARS = 300, 5

sweep_rows = []
for market, mult in MARKET_MULTS.items():
    for ret in RETENTIONS:
        program = create_layers(ret, mult)
        premium = float(program.calculate_premium())
        roe_list, ruins = [], 0
        for _ in range(N_SIMS):
            m = WidgetManufacturer(config)
            init_eq = float(m.equity)
            survived = True
            for _ in range(N_YEARS):
                _, stats = loss_generator.generate_losses(1.0, float(m.calculate_revenue()))
                recovery = program.process_claim(stats["total_amount"])
                net = stats["total_amount"] - recovery["insurance_recovery"]
                if net > 0:
                    m.process_insurance_claim(net)
                if premium > 0:
                    m.record_insurance_premium(premium)
                m.step(growth_rate=0.0)
                if float(m.equity) <= 0:
                    ruins += 1; survived = False; break
            if survived:
                roe_list.append((float(m.equity) / init_eq) ** (1 / N_YEARS) - 1)
        sweep_rows.append({
            "market": market, "retention": ret, "premium": premium,
            "mean_roe": np.mean(roe_list) if roe_list else -1,
            "ruin_prob": ruins / N_SIMS,
        })

sweep_df = pd.DataFrame(sweep_rows)
print(f"Sweep complete: {len(sweep_df)} points")
sweep_df.head(6)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
for ax, metric, label in zip(axes,
    ["mean_roe", "ruin_prob", "premium"],
    ["Mean ROE", "Ruin Probability", "Annual Premium ($)"],
):
    for mkt in MARKET_MULTS:
        sub = sweep_df[sweep_df["market"] == mkt]
        ax.plot(sub["retention"], sub[metric], "o-", label=mkt)
    ax.set_xlabel("Retention ($)")
    ax.set_ylabel(label)
    ax.legend()
    ax.grid(True, alpha=0.3)

axes[0].set_title("ROE vs. Retention")
axes[1].set_title("Ruin Probability vs. Retention")
axes[2].set_title("Premium vs. Retention")
plt.suptitle("Retention Sweep by Market Cycle", fontweight="bold")
plt.tight_layout()
plt.show()

# Report optimal retention per market
for mkt in MARKET_MULTS:
    sub = sweep_df[(sweep_df["market"] == mkt) & (sweep_df["ruin_prob"] < 0.05)]
    if len(sub):
        best = sub.loc[sub["mean_roe"].idxmax()]
        print(f"{mkt:>6s} market  optimal retention=${best['retention']:,.0f}  "
              f"ROE={best['mean_roe']:.2%}  ruin={best['ruin_prob']:.1%}")

## 3. Ergodic vs. Ensemble Perspective

Compare time-average (ergodic) growth with the ensemble mean to illustrate why
the optimal retention differs under each lens.

In [ ]:
# Pick optimal retention from normal market
normal_sub = sweep_df[(sweep_df["market"] == "Normal") & (sweep_df["ruin_prob"] < 0.05)]
OPT_RET = int(normal_sub.loc[normal_sub["mean_roe"].idxmax(), "retention"])

N_TRAJ, T_YEARS = 200, 15
program = create_layers(OPT_RET)
premium = float(program.calculate_premium())

trajectories = np.zeros((N_TRAJ, T_YEARS + 1))
trajectories[:, 0] = config.initial_assets

for s in range(N_TRAJ):
    m = WidgetManufacturer(config)
    for yr in range(T_YEARS):
        _, stats = loss_generator.generate_losses(1.0, float(m.calculate_revenue()))
        recovery = program.process_claim(stats["total_amount"])
        net = stats["total_amount"] - recovery["insurance_recovery"]
        if net > 0:
            m.process_insurance_claim(net)
        if premium > 0:
            m.record_insurance_premium(premium)
        m.step(growth_rate=0.0)
        trajectories[s, yr + 1] = max(float(m.total_assets), 0)

ensemble_mean = trajectories.mean(axis=0)
log_mean = np.exp(np.mean(np.log(np.clip(trajectories, 1, None)), axis=0))  # geometric / ergodic

print(f"Optimal retention: ${OPT_RET:,.0f}")
print(f"Ensemble mean final assets : ${ensemble_mean[-1]:>14,.0f}")
print(f"Ergodic (geometric) final  : ${log_mean[-1]:>14,.0f}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Trajectory fan
ax = axes[0]
t = np.arange(T_YEARS + 1)
for s in range(min(50, N_TRAJ)):
    ax.plot(t, trajectories[s] / 1e6, color="steelblue", alpha=0.1, lw=0.6)
ax.plot(t, ensemble_mean / 1e6, "r-", lw=2, label="Ensemble mean")
ax.plot(t, log_mean / 1e6, "g--", lw=2, label="Ergodic (geometric) mean")
ax.set_xlabel("Year")
ax.set_ylabel("Total Assets ($M)")
ax.set_title("Wealth Trajectories")
ax.legend()
ax.grid(True, alpha=0.3)

# Growth rate distributions
ax = axes[1]
final = trajectories[:, -1]
surviving = final[final > 0]
if len(surviving) > 1:
    growth_rates = (surviving / config.initial_assets) ** (1 / T_YEARS) - 1
    ax.hist(growth_rates * 100, bins=30, color="steelblue", alpha=0.7, edgecolor="white")
    ax.axvline(np.mean(growth_rates) * 100, color="r", ls="--", label=f"Mean={np.mean(growth_rates):.2%}")
    ax.axvline(np.median(growth_rates) * 100, color="g", ls="--", label=f"Median={np.median(growth_rates):.2%}")
    ax.set_xlabel("Annualized Growth Rate (%)")
    ax.set_ylabel("Count")
    ax.set_title("Growth Rate Distribution")
    ax.legend()
ax.grid(True, alpha=0.3)

plt.suptitle(f"Ergodic vs Ensemble -- Retention ${OPT_RET:,.0f}", fontweight="bold")
plt.tight_layout()
plt.show()

## 4. Multi-Year Market-Cycle Simulation

Simulate a company navigating market-cycle transitions over 20 years, re-pricing
insurance each year based on the prevailing cycle.

In [ ]:
import random

CYCLE_NAMES = ["Soft", "Normal", "Hard"]
CYCLE_MULTS = [0.80, 1.00, 1.40]
# Markov transition matrix: rows=from, cols=to
TRANS = np.array([
    [0.6, 0.3, 0.1],  # Soft  -> ...
    [0.2, 0.6, 0.2],  # Normal -> ...
    [0.1, 0.3, 0.6],  # Hard  -> ...
])

random.seed(SEED); np.random.seed(SEED)
SIM_YEARS = 20
N_PATHS = 100

market_paths = np.zeros((N_PATHS, SIM_YEARS), dtype=int)
asset_paths = np.zeros((N_PATHS, SIM_YEARS + 1))
asset_paths[:, 0] = config.initial_assets

for p in range(N_PATHS):
    state = 1  # start Normal
    m = WidgetManufacturer(config)
    for yr in range(SIM_YEARS):
        state = np.random.choice(3, p=TRANS[state])
        market_paths[p, yr] = state
        program = create_layers(OPT_RET, CYCLE_MULTS[state])
        prem = float(program.calculate_premium())
        _, stats = loss_generator.generate_losses(1.0, float(m.calculate_revenue()))
        recovery = program.process_claim(stats["total_amount"])
        net = stats["total_amount"] - recovery["insurance_recovery"]
        if net > 0:
            m.process_insurance_claim(net)
        if prem > 0:
            m.record_insurance_premium(prem)
        m.step(growth_rate=0.0)
        asset_paths[p, yr + 1] = max(float(m.total_assets), 0)

ruin_rate = np.mean(asset_paths[:, -1] <= 0)
surviving_final = asset_paths[:, -1][asset_paths[:, -1] > 0]
print(f"20-year ruin rate : {ruin_rate:.1%}")
if len(surviving_final):
    cagr = (surviving_final / config.initial_assets) ** (1 / SIM_YEARS) - 1
    print(f"Median CAGR       : {np.median(cagr):.2%}")
    print(f"Mean   CAGR       : {np.mean(cagr):.2%}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Asset paths
ax = axes[0]
t = np.arange(SIM_YEARS + 1)
for p in range(min(40, N_PATHS)):
    ax.plot(t, asset_paths[p] / 1e6, alpha=0.15, lw=0.6, color="steelblue")
ax.plot(t, np.median(asset_paths, axis=0) / 1e6, "g-", lw=2, label="Median")
ax.plot(t, np.mean(asset_paths, axis=0) / 1e6, "r--", lw=2, label="Mean")
ax.set_xlabel("Year")
ax.set_ylabel("Total Assets ($M)")
ax.set_title("20-Year Asset Paths with Market Transitions")
ax.legend()
ax.grid(True, alpha=0.3)

# Market state distribution over time
ax = axes[1]
for idx, name in enumerate(CYCLE_NAMES):
    frac = (market_paths == idx).mean(axis=0)
    ax.plot(range(SIM_YEARS), frac, "o-", label=name)
ax.set_xlabel("Year")
ax.set_ylabel("Fraction of Paths")
ax.set_title("Market Cycle Distribution Over Time")
ax.legend()
ax.grid(True, alpha=0.3)

plt.suptitle("Multi-Year Market-Cycle Simulation", fontweight="bold")
plt.tight_layout()
plt.show()

## Key Takeaways

- **Optimal retention rises in hard markets** because premium savings from higher deductibles are magnified when rates are elevated.
- The **ergodic (geometric) mean** grows more slowly than the ensemble mean -- a direct illustration of why time-average optimization differs from expectation-based approaches.
- **Market-cycle transitions** create path-dependent outcomes; strategies must be robust across regimes, not just optimal for a single cycle.
- A Markov model for cycle transitions captures realistic autocorrelation in pricing environments.

## Next Steps

- [optimization/05_parameter_sweeps](05_parameter_sweeps.ipynb) -- grid-search the full parameter space
- [optimization/02_sensitivity_analysis](02_sensitivity_analysis.ipynb) -- tornado diagrams and interaction effects
- [optimization/03_pareto_analysis](03_pareto_analysis.ipynb) -- multi-objective trade-off frontiers